In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import os

In [32]:
data_path = '../sensor_data/'
os.listdir(data_path)

['s1746788', 's1701688', '.DS_Store', 's1721039', 's1711507', 's1660711']

In [33]:
users = ['s1746788', 's1701688', 's1721039', 's1711507', 's1660711']
full_paths = [data_path + usr + '/' + filename for usr in users for filename in os.listdir(data_path + usr)]
header_size = 5

In [34]:
class DataRecording:  
    def __init__(self, sensor_position, sensor_side, activity, activity_id):
        self.sensor_position = sensor_position
        self.sensor_side = sensor_side
        self.activity = activity
        self.activity_id = activity_id
    
    @classmethod
    def create_recording(cls, filepath):
        with open(filepath) as f:
            head = [next(f).rstrip().split('# ')[1].split(':')[1].strip() for x in range(header_size)]
            recording = DataRecording(head[0], head[1], head[2], head[3])
        df = pd.read_csv(filepath, header=5)
        recording.data = df
        return recording   

In [35]:
all_data = []
full_paths
for path in full_paths:
    try:
        all_data.append(DataRecording.create_recording(path))
    except:
        pass
#         print(path)

In [36]:
len(all_data)

286

In [37]:
allowed_activites = ['Climbing stairs', 'Descending stairs', 'Walking at normal speed', 'Running', 'Standing']
right_pocket_data = [x for x in all_data if 
                        x.sensor_position == 'Front pocket (trouser)' 
                            and 
                        x.sensor_side == 'Right'
                           and
                        x.activity in allowed_activites
                       ]
len(right_pocket_data)

18

In [38]:
def get_overlapping_windows(df, win_size, overlap, extract_features):
    rows,  = df.shape
    assert(overlap < 1)
    assert(overlap >= 0)
    assert(win_size <= rows)

    increment = int(win_size * (1 - overlap))
    result = []
    for i in range(0, rows - win_size, increment):
        feature_vector = extract_features(df[i:i+win_size])
        result.append(feature_vector)
    return np.array(result)


In [39]:
def get_feature_vector(df):
    return np.array([df.mean(), df.max() - df.min(), df.var(), df.std()])

In [40]:
def process_data(data_objects):
    df = pd.DataFrame()
    for d_o in data_objects:
        processed_windows = get_overlapping_windows(d_o.data['accel_x'], 50, 0.5, get_feature_vector)
        tmp_df = pd.DataFrame(processed_windows)
        tmp_df['Activity'] = d_o.activity_id
        df = pd.concat([df, tmp_df])
    return df
        

In [41]:
full_df = process_data(right_pocket_data)
full_df.shape

(351, 5)

In [42]:
from sklearn import svm

In [43]:
X = full_df.iloc[:, :4]
X.shape

(351, 4)

In [50]:
y = np.array(full_df.iloc[:, 4:]).reshape((351,))
y.shape

(351,)

In [52]:
clf = svm.SVC()
clf.fit(X, y)

SVC()

In [60]:
y_preds = clf.predict(X)

In [64]:
accuracy = (np.sum(y_preds == y)*100)/y_preds.shape[0]
print(accuracy)

65.52706552706553
